In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('./data/sales_train.csv').dropna()[['unique_id', 'date', 'sales', 'warehouse', 'total_orders', 'sell_price_main'] \
                                                    + [f'type_{i}_discount' for i in range(7)]]
df['date'] = pd.to_datetime(df['date'])
df = df.loc[df['date'] >= '06-01-2022']

df['city'] = df['warehouse'].apply(lambda x: x.split('_')[0])
df['dayofweek'] = df['date'].dt.dayofweek
df['year'] = df['date'].dt.year

warehouse_weight = {
    'Brno_1': 0.060415,
    'Budapest_1': 0.002977,
    'Frankfurt_1': 1.355029,
    'Munich_1': 1.762047,
    'Prague_1': 0.066596,
    'Prague_2': 0.038047,
    'Prague_3': 0.031790
}

df['ware_wgt'] = df['warehouse'].apply(lambda x: warehouse_weight[x])

df['sales'] = df['sell_price_main'] * df['sales'] * df['ware_wgt']

df['discount_amount'] = df['sell_price_main'] * df[[f'type_{i}_discount' for i in range(7)]].max(axis=1)

In [2]:
t_cols = ['sales', 'date', 'unique_id']
data = df[t_cols].copy()
data['date'] = pd.to_datetime(data['date'])
data['month'] = data['date'].dt.to_period('M')

# prev_month average
monthly_avg = (
        data.groupby(['unique_id', 'month'])['sales']
        .mean()
        .groupby('unique_id')
        .shift(1)
        .reset_index()
        .rename(columns={'sales': 'prev_month_avg'})
    )
data = data.merge(monthly_avg, on=['unique_id', 'month'], how='left').fillna(0)
data.drop(columns=['month', 'sales'], inplace=True)

df = df.merge(data, how='left', on=['unique_id', 'date'])

# robust week encoding
week_encoding = df[['dayofweek', 'sales', 'city', 'year']].groupby(
        ['year', 'dayofweek', 'city']).mean().reset_index()
week_mean = df[['sales', 'city', 'year']].groupby(
        ['year', 'city']).mean().reset_index()
week_mean.rename(columns={'sales': 'mean_sales'}, inplace=True)
week_encoding.rename(columns={'sales': 'weekday_avg_sales'}, inplace=True)
week_encoding = week_encoding.merge(
        week_mean, how='left', on=['city', 'year'])
week_encoding['weekday_frac_sales'] = week_encoding['weekday_avg_sales'] / \
        week_encoding['mean_sales']

week_encoding.drop('mean_sales', axis=1, inplace=True)

df = df.merge(week_encoding, how='left', on=[
                        'year', 'dayofweek', 'city'])

df['week_trend'] = df['weekday_frac_sales'] * df['prev_month_avg']

In [3]:
# lags
PERIODS = [14, 21, 28, 35]

df = df.sort_values(['unique_id', 'date'])
for shift in PERIODS:
    df[f'product_sales_{shift}'] = (df.groupby('unique_id', observed=True)['sales']
                                    .transform(lambda x: x.shift(shift).fillna(0))
                                    )

df['moving'] = (df.groupby('unique_id')['sales']
                    .transform(lambda x: x.shift(14).rolling(window=14, min_periods=1).mean().fillna(0))
                    )
df['week_moving_trend'] = df['weekday_frac_sales'] * df['moving']

# lags
DAY_PERIODS = [14, 21, 28, 35]
OFF_PERIODS = [15, 16, 17, 18, 19, 20]
orders = df['total_orders']
for shift in DAY_PERIODS:
    grouped = df[['unique_id', 'sales', 'total_orders']].groupby('unique_id')
    sales = grouped['sales'].transform(lambda x: x.shift(shift))
    s_orders = grouped['total_orders'].transform(lambda x: x.shift(shift))
    df[f'lag_{shift}'] = (orders * sales / s_orders).fillna(0)

numerator = df['weekday_frac_sales'].shift(periods=14)
for shift in OFF_PERIODS:
    grouped = df[['unique_id', 'sales', 'weekday_frac_sales']].groupby('unique_id')
    sales = grouped['sales'].transform(lambda x: x.shift(shift))
    frac = grouped['weekday_frac_sales'].transform(
        lambda x: x.shift(shift))
    df[f'lag_{shift}'] = (numerator * sales / frac).fillna(0)

df['normed_week_mean'] = df[['lag_14'] + [f'lag_{i}' for i in DAY_PERIODS]].mean(axis=1)
df['normed_week_median'] = df[['lag_14'] + [f'lag_{i}' for i in DAY_PERIODS]].median(axis=1)

df = df.drop([f'lag_{i}' for i in OFF_PERIODS], axis=1)

In [4]:
import cvxpy as cp
from tqdm import tqdm

nec_cols = ['product_sales_14', 'product_sales_21', 'product_sales_28', 'product_sales_35',
            'weekday_avg_sales', 'week_trend', 'week_moving_trend', 'moving', 'normed_week_mean',
            'normed_week_median', 'discount_amount'] + [f'lag_{i}' for i in [14, 21, 28, 35]]

def mse_reg(X: pd.DataFrame, y: pd.Series):
    X_np = X.to_numpy()
    y_np = y.to_numpy()

    _, d = X_np.shape
    beta = cp.Variable(d)
    intercept = cp.Variable()

    residuals = y_np - (X_np @ beta + intercept)
    objective = cp.Minimize(cp.sum(cp.sum_squares(residuals)))
    problem = cp.Problem(objective)
    problem.solve()

    return beta.value, intercept.value

for col in nec_cols + ['sales']:
    df[col] = np.sqrt(df[col])

In [5]:
size = df[['unique_id', 'date']].groupby('unique_id', observed=True).count(
).reset_index().rename(columns={'date': 'n_many'})
small = size.loc[size['n_many'] < 60]
big = size.loc[size['n_many'] >= 60]
lil_df = df.merge(small, how='inner', on='unique_id')[
    ['unique_id', 'sales'] + nec_cols]
big_df = df.merge(big, how='inner', on='unique_id')[
    ['unique_id', 'sales'] + nec_cols]

disc_values = big_df[['unique_id', 'discount_amount']].groupby('unique_id', observed=True).count(
).reset_index().rename(columns={'discount_amount' : 'n_many'})

no_disc_ids = disc_values.loc[disc_values['n_many'] <= 1, 'unique_id'].to_list()
disc_ids = disc_values.loc[disc_values['n_many'] > 1, 'unique_id'].to_list()

lil_ids = lil_df['unique_id'].unique().tolist()

dict = {}
for id in tqdm(disc_ids):
    beta, intercept = mse_reg(
        big_df.loc[big_df['unique_id'] == id, nec_cols], big_df.loc[big_df['unique_id'] == id, 'sales'])
    dict[id] = {
        'unique_id': id,
        **{'coef_' + col: coef for col, coef in zip(nec_cols, beta)},
        'intercept': intercept
    }

no_disc = nec_cols.copy()
no_disc.remove('discount_amount')

for id in tqdm(no_disc_ids):
    beta, intercept = mse_reg(
        big_df.loc[big_df['unique_id'] == id, no_disc], big_df.loc[big_df['unique_id'] == id, 'sales'])
    dict[id] = {
        'unique_id': id,
        **{'coef_' + col: coef for col, coef in zip(no_disc, beta)},
        'coef_discount_amount': 0,
        'intercept': intercept
    }

smol_cols = ['weekday_avg_sales']
non_cols = nec_cols.copy()
for col in smol_cols:
    non_cols.remove(col)
for id in tqdm(lil_ids):
    beta, intercept = mse_reg(
        lil_df.loc[lil_df['unique_id'] == id, smol_cols], lil_df.loc[lil_df['unique_id'] == id, 'sales'])
    dict[id] = {
        'unique_id': id,
        **{'coef_' + col: coef for col, coef in zip(smol_cols, beta)},
        **{'coef_' + col: 0 for col in non_cols},
        'intercept': intercept
    }

dataframe = pd.DataFrame(dict).T
dataframe['unique_id'] = dataframe['unique_id'].astype(int)
display(dataframe)
dataframe.to_csv('./trend_coefs.csv', index=False)

100%|██████████| 4709/4709 [07:59<00:00,  9.82it/s]
0it [00:00, ?it/s]
100%|██████████| 535/535 [00:02<00:00, 190.46it/s]


,unique_id,coef_product_sales_14,coef_product_sales_21,coef_product_sales_28,coef_product_sales_35,coef_weekday_avg_sales,coef_week_trend,coef_week_moving_trend,coef_moving,coef_normed_week_mean,coef_normed_week_median,coef_discount_amount,coef_lag_14,coef_lag_21,coef_lag_28,coef_lag_35,intercept
1,1,0.535454,-0.355344,-0.242492,0.443893,3.128062,-0.01183,-1.649108,1.76064,0.083058,0.186195,4.927988,-0.593449,0.319674,0.229853,-0.419658,-25.311273277539843
3,3,-0.235871,-0.036021,-0.040545,0.0291,-1.023609,-0.08894,1.117969,-0.829153,-1.69167,0.516257,0.293454,0.917668,0.320144,0.449123,0.348009,18.77463791575167
5,5,-0.088764,-0.290892,0.155013,-0.279722,0.320421,0.003669,0.162416,-0.320164,0.68861,0.333132,-0.0,-0.190618,0.132083,-0.20673,0.098267,6.2280792560959055
6,6,-0.080364,-0.132805,-0.072882,-0.045432,0.300056,-0.328002,0.397064,-0.370647,0.58929,0.160892,7.613631,-0.050779,0.012857,0.060814,-0.002085,6.16813818079642
7,7,-0.020719,-0.056633,-0.10633,-0.247133,0.002416,-0.161724,0.713608,-0.420517,-0.593137,0.244142,1.088961,0.31245,0.176907,0.284319,0.483924,2.7101111131436473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5372,5372,0,0,0,0,-0.514215,0,0,0,0,0,0,0,0,0,0,22.128418717333012
5390,5390,0,0,0,0,1.881407,0,0,0,0,0,0,0,0,0,0,-12.525476191799108
5416,5416,0,0,0,0,18.002294,0,0,0,0,0,0,0,0,0,0,-200.23841243948672
5417,5417,0,0,0,0,0.352305,0,0,0,0,0,0,0,0,0,0,17.953265516828036
